In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Clustering Analysis").getOrCreate()

In [3]:
# Read in files with pandas
user_artist_data = pd.read_csv('data/user_artist_data_small.txt', names=['user_id', 'artist_id', 'count'], sep=' ')
artist_data = pd.read_csv('data/artist_data_small.txt', names=['artist_id', 'artist_name'], sep='\t')
artist_alias = pd.read_csv('data/artist_alias_small.txt', names=['alias_id', 'artist_id'], sep='\t')

In [4]:
# Create RDD
uad = spark.createDataFrame(user_artist_data)

In [5]:
# Look at uad_df
print(uad.select('artist_id').rdd.min())
print(uad.select('artist_id').rdd.max())

Row(artist_id=1)
Row(artist_id=10788218)


In [6]:
# Create a dictionary of aliases
alias_dict = artist_alias.set_index('alias_id')['artist_id'].to_dict()

In [7]:
def unalias(x):
    if x['artist_id'] in alias_dict:
        x['artist_id'] = alias_dict[x['artist_id']]
    return x

uad = uad.rdd.map(unalias)
uad.take(2)

[Row(user_id=1059637, artist_id=1000010, count=238),
 Row(user_id=1059637, artist_id=1000049, count=1)]

In [20]:
def make_rating(x):
    return Rating(int(x['user_id']), int(x['artist_id']), int(x['count']))

ratings = uad.map(make_rating)

In [21]:
print(type(ratings))
print(type(ratings.take(1)[0]))
ratings = ratings.map(lambda row: (row[0], row[1], row[2]))
ratings.take(2)
print(type(ratings.take(1)[0]))

<class 'pyspark.rdd.PipelinedRDD'>
<class 'pyspark.mllib.recommendation.Rating'>
<class 'tuple'>


In [11]:
ratings_df = ratings.toDF()
ratings_df

DataFrame[_1: bigint, _2: bigint, _3: double]

In [ ]:
training, test = rating_df = 

In [13]:
type(ratings)

pyspark.rdd.PipelinedRDD

In [22]:
# Build the recommendation model using Alternating Least Squares
# ALS train arguments: ratings, rank, iterations=5, lambda_=0.01, blocks=-1
rank = 10
iterations = 5
lambda_ = 0.01
blocks = 1.0
model = ALS.trainImplicit(ratings, rank, iterations, alpha=0.01)

Py4JJavaError: An error occurred while calling o453.trainImplicitALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 35.0 failed 1 times, most recent failure: Lost task 1.0 in stage 35.0 (TID 44, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-7-16f4e708da9f>", line 3, in unalias
TypeError: 'Row' object does not support item assignment

	

In [10]:
# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

NameError: name 'model' is not defined

In [10]:
# Map other artist names to their correct IDs
def unalias(x):
    if x in alias_dict:
        return alias_dict[x]
    else:
        return x

mapped = uad.select('artist_id').rdd.map(unalias)
print(mapped)
print(mapped.toDF()['artist_id'])
uad_dropped = uad.drop('artist_id')
print(uad_dropped.show())
uad_new = uad_dropped.withColumn('new_artist_id', mapped.toDF()['artist_id'])
mapped.take(4)

PythonRDD[39] at RDD at PythonRDD.scala:48
Column<b'artist_id'>
+-------+-----+
|user_id|count|
+-------+-----+
|1059637|  238|
|1059637|    1|
|1059637|    1|
|1059637|   11|
|1059637|    1|
|1059637|  423|
|1059637|    5|
|1059637|    2|
|1059637|    2|
|1059637|19129|
|1059637|    4|
|1059637|  188|
|1059637|  180|
|1059637|    2|
|1059637|    1|
|1059637|   21|
|1059637|    1|
|1059637|   20|
|1059637|   12|
|1059637|   10|
+-------+-----+
only showing top 20 rows

None


AnalysisException: 'resolved attribute(s) artist_id#42L missing from user_id#23L,count#25L in operator !Project [user_id#23L, count#25L, artist_id#42L AS new_artist_id#45L];;\n!Project [user_id#23L, count#25L, artist_id#42L AS new_artist_id#45L]\n+- Project [user_id#23L, count#25L]\n   +- LogicalRDD [user_id#23L, artist_id#24L, count#25L]\n'

In [96]:
# Explore alias df
print(len(artist_alias['artist_id'].unique()))
print(len(artist_alias['alias_id'].unique()))

486
587


In [74]:
# There are more alias_id records than there are artist_id records
# So lets take the user_artist_data, and change every artist id to it's "un-aliased" id
def unalias(x):
    try:
        unaliased_id = artist_alias[artist_alias['alias_id'] == x]['artist_id'][0]
        return unaliased_id
    except:
        return x

user_artist_data['correct_artist_id'] = user_artist_data['artist_id'].apply(unalias)

In [90]:
# Another way that might be faster
user_artist_data['old_artist_id'] = user_artist_data['artist_id']
unaliased_uad = user_artist_data.set_index('artist_id').join(artist_alias.set_index('alias_id'))
unaliased_uad['artist_id'] = unaliased_uad['artist_id'].fillna(unaliased_uad['old_artist_id'])
unaliased_uad = unaliased_uad.drop('old_artist_id', axis=1)
unaliased_uad

,user_id,count,artist_id
1,1046559,147,1.0
1,1052461,3481,1.0
1,1031009,4,1.0
1,2062243,22,1.0
1,1058890,3,1.0
1,2288164,4006,1.0
1,1063644,331,1.0
1,1048402,53,1.0
1,1017610,376,1.0
1,2023686,49,1.0


In [81]:
user_artist_data.head()

,user_id,artist_id,count,correct_artist_id
0,1059637,1000010,238,1000010
1,1059637,1000049,1,1000049
2,1059637,1000056,1,1000056
3,1059637,1000062,11,1000062
4,1059637,1000094,1,1000094


In [42]:
user_artist_data = spark.sparkContext.textFile('data/user_artist_data_small.txt').map(lambda x: x.split())
artist_data = spark.sparkContext.textFile('data/artist_data_small.txt').map(seperate_artist_data)

PythonRDD[50] at RDD at PythonRDD.scala:48

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 22, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-38-4ef1c3b2bad4>", line 3, in seperate_artist_data
ValueError: not 